什么是LCEL
    从原型到生产，完整流程不需要修改任何代码，也就是我们在写的任何原型
    代码不需要太多的改变就能支持生产级别的各种特性比如并行 streaming等，具体会有这些优势
     * 自动并行
     * 自动重试和fallback
     * 对chain中间结果的访问
     * LCEL会自动支持LangSimith进行可视化和记录，可以记录一条chain运行过程中的大部分信息

 一条chain组成的每个模块都是继承自Runnable这个接口，而每一条chain也是继承自这个接口，所以一条Chain也可以很自然的成为另一个Chain的一个模块
 并且所有的Runable都有相同的调用方式,所以我们在写Chain的时候就可以自由组合多个Runnable的模块来形成复杂的Chain

*   invoke 基础的调用，并传入参数
*   batch 批量调用，输入一组参数
*   stream 调用，并以stream流的方式返回数据
*   stremLog 除了像stream流一样返回数据，并会返回中间的运行结果

In [7]:
import { load } from "https://deno.land/std@0.223.0/dotenv/mod.ts";
import { ChatOpenAI } from "@langchain/openai";
import { HumanMessage } from "@langchain/core/messages";
import { StringOutputParser } from "@langchain/core/output_parsers";

const env = await load();

const process = {
    env
}


const chatModel = new ChatOpenAI({
    model:'gpt-3.5-turbo',
    configuration:{
        baseURL: "https://ai.nvtnet.net/v1",
    }
});

const outputParse = new StringOutputParser();

//用pipe组成一个最基本的chain
const simpleChain = chatModel.pipe(outputParse);

// const res = await simpleChain.invoke(
//    [ new HumanMessage("tell me a joke")]
// )

const stream = await simpleChain.stream([
    new HumanMessage("Tell me a joke")
])

for await (const chunk of stream){
   console.log(chunk)
}


Why
 couldn
't
 the
 bicycle
 stand
 up
 by
 itself
?


Because
 it
 was
 two
-t
ired
!



在 LCEL 中，使用 .pipe() 方法来组装多个 Runnable 对象形成完整的 Chain，可以看到我们是用对单个模块同样的 invoke 方法去调用整个 chain。 因为无论是单个模块还是由模块组装而成的多个 chain 都是 Runnable